# Heterologous pathway addition and importing the model

## importing the model

In [3]:
from cobra.io import read_sbml_model
model = read_sbml_model('/Users/wagner/github-classroom/27410/group-assingment-team-4/yeast-GEM-8.7.0/model/yeast-GEM.xml') # loading
from cobra import Reaction, Metabolite, Gene

In [4]:
model

Name,yeastGEM_v8__46__7__46__0
Memory address,28ec2efd0
Number of metabolites,2807
Number of reactions,4133
Number of genes,1163
Number of groups,106
Objective expression,1.0*r_2111 - 1.0*r_2111_reverse_58b69
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


Since there are no production pathways inherently in the model of the organism we will need to add the metabolite and reaction ourselves

Adding the metabolites:

In [18]:
Beta_farnesene = Metabolite(name='Beta-farnesene',id='beta_farnesene_c',compartment='c')
# Created the metabolite Beta-farnesene as a metabolite that is present in the cytoplasm

Adding the reaction for beta-farnesene and the demand reaction for beta-farnesene

In [19]:
from cobra import Reaction, Metabolite, Gene

Beta_farnesene_synthase_reaction = Reaction('R08695') # Creates the Beta-farnesene reaction 

# This creates the reaction for beta-farnesene synthesis by indicating the consumption of the substrates and 
# using our newly introduced metabolite as the product
# The reaction R08695 is:
# Trans,trans-Farnesyl diphosphate <=> beta-Farnesene + Diphosphate
# These are found as the metabolites:
# Farnesyl diphosphate = s_0190
# beta-farnesene = Beta_farnesene
# Diphosphate = s_0633 

# Thus having already defined the metabolite and the reaction we can fully define the reaction stoichiometry:

Beta_farnesene_synthase_reaction.add_metabolites({model.metabolites.s_0190: -1,
                                                  model.metabolites.s_0633: 1,
                                                  Beta_farnesene: 1 # our product
                                                  }) 

# Test if the reaction has been made properly and the stoichiometric definition is correct:
Beta_farnesene_synthase_reaction

# Adding the reaction to the model
model.add_reactions([Beta_farnesene_synthase_reaction])

# tests that the reaction has been added to the model
model.reactions.get_by_id(Beta_farnesene_synthase_reaction.id)

Ignoring reaction 'R08695' since it already exists.


Reaction identifier,R08695
Name,
Memory address,0x298f37b90
Stoichiometry,s_0190 --> beta_farnesene_c + s_0633 farnesyl diphosphate --> Beta-farnesene + diphosphate
GPR,
Lower bound,0.0
Upper bound,1000.0


In [20]:
# This creates the demand reaction for beta-farnesene as this otherwise would become a dead-end metabolite:

# The demand reaction follows the same framework as above, with the stoichiometry just being -1 for the
# metabolite Beta-farnesene:

Beta_farnesene_demand_reaction = Reaction('BFD')
Beta_farnesene_demand_reaction.add_metabolites({
                                                  Beta_farnesene: -1 # our product
                                                  })
model.add_reactions([Beta_farnesene_demand_reaction])

# Note: the output is because we already added the reaction to the model.

Ignoring reaction 'BFD' since it already exists.


In [21]:
# tests that the reaction has been added to the model
model.reactions.get_by_id(Beta_farnesene_demand_reaction.id)



Reaction identifier,BFD
Name,
Memory address,0x298f43410
Stoichiometry,beta_farnesene_c --> Beta-farnesene -->
GPR,
Lower bound,0.0
Upper bound,1000.0


## Testing to make sure flux goes through the production pathway that we have created

In [34]:
# This function test to see that the organism still is growing
with model:
   print(model.optimize().objective_value)

# now we will set the objective function of the model to the demand reaction for beta-farnesene to test if
# the model allows flux through the beta-farnsene biosynthesis pathway
with model:
   model.objective = model.reactions.BFD
   print('This is the maximal production of beta-farnesene on glucose:', model.optimize().objective_value, 'mmol beta-farnesene gDW^-1 h^-1')

0.08192818581102052
This is the maximal production of beta-farnesene on glucose: 0.23035112219719903 mmol beta-farnesene gDW^-1 h^-1


It is evident that we are seing flux through the reaction when setting it as the model objective function, this means that it is not a dead end metabolite.

# Saving the model:

In [33]:
# This code saves the changes to the model and we now have a model that has a the heterologous pathway for 
# beta-farnesene production as well as a demand reaction for cytoplasmic beta-farnesene

io.write_yeast_model(model)   # saving

# Finding the most suitable carbon source and testing different types of media

First we can test the production of beta-farnesene in aerobic conditions as well as anaerobic conditions:

In [61]:
model.reactions.query('oxygen exchange', 'name')


[<Reaction r_1992 at 0x28ffe7710>]

In [64]:
with model:
   medium = model.medium
   medium['r_1992'] = 0
   model.medium = medium
   with model:
      model.objective = model.reactions.BFD
      BF_production = model.optimize().objective_value
print(BF_production)

0.047392290249433075


There is no production in this case no point in looking further at that, so we will go back to aerobic conditions:

In [68]:
with model:
    model.objective = model.reactions.BFD
    BF_production = model.optimize().objective_value
    print("Maximum beta-farnesene production [mmol-beta-farnesene gDW^-1 h^-1]:", BF_production)
    print("Theoretical maximum yield [mmol-beta-farnesene / mmol-glucose]:", BF_production / (-1*model.reactions.r_1714.flux))
    maximum_yield_mmol=BF_production / (-1*model.reactions.r_1714.flux)

Maximum beta-farnesene production [mmol-beta-farnesene gDW^-1 h^-1]: 0.23035112219719758
Theoretical maximum yield [mmol-beta-farnesene / mmol-glucose]: 0.23035112219719758


Now we will test to see the production of beta-farnesene with increased glucose concentrations:

In [6]:
# First we will here make an overview of the full medium composition as it is in the model by default with
# growth on glucose, to do this we check through all the extracellular exchange reactions:

for component in model.medium.items():
    print(model.reactions.get_by_id(component[0]), "|", model.reactions.get_by_id(component[0]).name, "| concentration =", component[1])

r_1654: s_0420 <=>  | ammonium exchange | concentration = 1000.0
r_1714: s_0565 <=>  | D-glucose exchange | concentration = 1.0
r_1832: s_0796 <=>  | H+ exchange | concentration = 1000.0
r_1861: s_0925 <=>  | iron(2+) exchange | concentration = 1000.0
r_1992: s_1277 <=>  | oxygen exchange | concentration = 1000.0
r_2005: s_1324 <=>  | phosphate exchange | concentration = 1000.0
r_2020: s_1374 <=>  | potassium exchange | concentration = 1000.0
r_2049: s_1438 <=>  | sodium exchange | concentration = 1000.0
r_2060: s_1468 <=>  | sulphate exchange | concentration = 1000.0
r_2100: s_0805 <=>  | water exchange | concentration = 1000.0
r_4593: s_4200 <=>  | chloride exchange | concentration = 1000.0
r_4594: s_4201 <=>  | Cu2(+) exchange | concentration = 1000.0
r_4595: s_4202 <=>  | Mn(2+) exchange | concentration = 1000.0
r_4596: s_4203 <=>  | Zn(2+) exchange | concentration = 1000.0
r_4597: s_4204 <=>  | Mg(2+) exchange | concentration = 1000.0
r_4600: s_4199 <=>  | Ca(2+) exchange | concen

This is a full overview of the medium composition. Using this we will now check the impact of changing the concentration of glucose in the medium on production and on growth:

In [24]:
# Testing different glucose concentrations of 0.1X, 10X, 100X and 1000X the standard glucose concentration in the medium

medium = model.medium
with model:
   medium['r_1714'] = 0.1
   model.medium = medium
   point_one_X_glc_growth = model.optimize().objective_value
   model.objective = model.reactions.BFD
   point_one_X_glc_productivity = model.optimize().objective_value
   print('Maximal growth with 0.1X glucose:', point_one_X_glc_growth)
   print('Maximal production of beta-farnesene with 0.1X glucose:', point_one_X_glc_productivity)
   print('Theoretical maximum beta-farnesene production on 0.1X glucose:', model.optimize().objective_value / (-1*model.reactions.r_1714.flux))

print('')

with model:
   medium['r_1714'] = 10
   model.medium = medium
   ten_X_glc_growth = model.optimize().objective_value
   model.objective = model.reactions.BFD
   ten_X_glc_productivity = model.optimize().objective_value
   print('Maximal growth with 10X glucose:', ten_X_glc_growth)
   print('Maximal production of beta-farnesene with 10X glucose:', ten_X_glc_productivity)
   print('Theoretical maximum beta-farnesene production on 10X glucose:', model.optimize().objective_value / (-1*model.reactions.r_1714.flux))

print('')

with model:
   medium['r_1714'] = 100
   model.medium = medium
   hundred_X_glc_growth = model.optimize().objective_value
   model.objective = model.reactions.BFD
   hundred_X_glc_productivity = model.optimize().objective_value
   print('Maximal growth with 100X glucose:', hundred_X_glc_growth)
   print('Maximal production of beta-farnesene with 100X glucose:', hundred_X_glc_productivity)
   print('Theoretical maximum beta-farnesene production on 100X glucose:', model.optimize().objective_value / (-1*model.reactions.r_1714.flux))

print('')

with model:
   medium['r_1714'] = 1000
   model.medium = medium
   thousand_X_glc_growth = model.optimize().objective_value
   model.objective = model.reactions.BFD
   thousand_X_glc_productivity = model.optimize().objective_value
   print('Maximal growth with 1000X glucose:', thousand_X_glc_growth)
   print('Maximal production of beta-farnesene with 1000X glucose:', thousand_X_glc_productivity)
   print('Theoretical maximum beta-farnesene production on 1000X glucose:', model.optimize().objective_value / (-1*model.reactions.r_1714.flux))



   

Maximal growth with 0.1X glucose: 0.005401622831738066
Maximal production of beta-farnesene with 0.1X glucose: 0.01666211119003101
Theoretical maximum beta-farnesene production on 0.1X glucose: 0.16662111190008425

Maximal growth with 10X glucose: 0.8471930218951111
Maximal production of beta-farnesene with 10X glucose: 2.3416532799480834
Theoretical maximum beta-farnesene production on 10X glucose: 0.2341653279948143

Maximal growth with 100X glucose: 8.499842132127485
Maximal production of beta-farnesene with 100X glucose: 23.454674857455117
Theoretical maximum beta-farnesene production on 100X glucose: 0.23454674857455074

Maximal growth with 1000X glucose: 19.817859616711548
Maximal production of beta-farnesene with 1000X glucose: 76.91806853582563
Theoretical maximum beta-farnesene production on 1000X glucose: 0.11999593706928548


It seems that there is little to no benefit in the theoretical maximum of beta-farnesene production neither increasing nor decreasing the glucose concentrations.

As a follow-up part we will now look into changing the media composition to the well known and used YPED Medium that is most often used for growing yeast, instead of the minimal medium that the model uses. 